In [1]:
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

# example of random oversampling to balance the class distribution
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from matplotlib import pyplot
from tensorflow import keras


import tensorflow_addons as tfa
import tensorflow as tf

from scipy import stats
from imblearn.over_sampling import RandomOverSampler, SMOTE


2023-03-13 22:30:28.112368: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 22:30:28.496687: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-13 22:30:29.558757: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/guilherme/cpu-tensorflow-marcelo/nvidia-smi/envs/tf/lib/
2023-03-13 22:30:29.561437: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7';

In [2]:
# configs
EPOCHS = 5
BATCH_SIZE = 32

TIME_SERIES_SIZE = 4   # Determines the window size. Ex (4,9)
TIME_STEP_SHIFT  = 1   # Determines specifies the number of steps to move the window forward at each iteration.

baseFolder = "../data_2019_processed/"

# selected features
inputFeatures = ["activity","location","day_of_week",
                 "light","phone_lock","proximity",
                 "sound","time_to_next_alarm", "minutes_day"]
outputClasses = ["awake","asleep"]
#outputClasses = ["class"]

In [3]:
# y_test     = Array with real values
# yhat_probs = Array with predicted values
def printMetrics(y_test,yhat_probs):
    # predict crisp classes for test set deprecated
    #yhat_classes = model.predict_classes(X_test, verbose=0)
    #yhat_classes = np.argmax(yhat_probs,axis=1)
    yhat_classes = yhat_probs.round()
    # accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(y_test, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(y_test, yhat_classes)
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, yhat_classes)
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(y_test, yhat_classes)
    print('F1 score: %f' % f1)
    # kappa
    kappa = cohen_kappa_score(y_test, yhat_classes)
    print('Cohens kappa: %f' % kappa)
    # ROC AUC
    auc = roc_auc_score(y_test, yhat_probs)
    print('ROC AUC: %f' % auc)
    # confusion matrix
    print("\Confusion Matrix")
    matrix = confusion_matrix(y_test, yhat_classes)
    print(matrix)
    
    results = dict()
    results['accuracy'] = accuracy
    results['precision'] = precision
    results['recall'] = recall
    results['f1_score'] = f1
    results['cohen_kappa_score'] = kappa
    results['roc_auc_score'] = auc
    return results

def showGlobalMetrics(metrics):
    accuracy,precision,recall,f1_score,cohen_kappa_score,roc_auc_score = 0,0,0,0,0,0
    for metric in metrics:
        accuracy = accuracy + metric['accuracy']
        precision = precision + metric['precision']
        recall = recall + metric['recall']
        f1_score = f1_score + metric['f1_score']
        cohen_kappa_score = cohen_kappa_score + metric['cohen_kappa_score']
        roc_auc_score = roc_auc_score + metric['roc_auc_score']
        
    # mean
    size = len(metrics)
    print(size)
    accuracy = accuracy / size
    precision = precision / size
    recall = recall / size
    f1_score = f1_score / size
    cohen_kappa_score = cohen_kappa_score / size
    roc_auc_score = roc_auc_score / size
    
    #show:\
    print("accuracy: ",accuracy)
    print("precision: ",precision)
    print("recall: ",recall)
    print("f1_score: ",f1_score)
    print("cohen_kappa_score: ",cohen_kappa_score)
    print("roc_auc_score: ",roc_auc_score)
    
def transform_data_type(dataframe):
    
    # transform inputs
    for column in inputFeatures:
        if(dataframe[column].dtypes in ()):
            dataframe[column] = dataframe[column].astype('float32')
    
    # transform outputs
    #for column in outputClasses:
    #    if(dataframe[column].dtypes != object):
    #        dataframe[column] = dataframe[column].astype('float32')
    
    return dataframe

# one-hot encoding function
def transform_output_nominal_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data['awake']
    dataset['asleep'] = one_hot_encoded_data['asleep']
    
    return dataset

# one-hot encoding function
def transform_output_numerical_class_into_one_hot_encoding(dataset):
    # create two classes based on the single class
    one_hot_encoded_data = pd.get_dummies(dataset['class'])
    #print(one_hot_encoded_data)
    dataset['awake'] = one_hot_encoded_data[0]
    dataset['asleep'] = one_hot_encoded_data[1]
    
    return dataset


def create_dataset_time_series_with_one_output(X, y, window_time_steps=1, shift_step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - window_time_steps, shift_step):
        v = X.iloc[i:(i + window_time_steps)].values
        labels = y.iloc[i: i + window_time_steps]
        Xs.append(v)        
        ys.append(stats.mode(labels)[0][0])
        
    if len(y.columns) == 1:
        return np.array(Xs), np.array(ys).reshape(-1, 1)
    else:
        return np.array(Xs), np.array(ys).reshape(-1, len(y.columns))

In [4]:
#X_train = pd.read_csv(baseFolder+"train/allData-classification-numeric-normalized.csv")
X_test  = pd.read_csv(baseFolder+"test/allData-classification-numeric-normalized.csv")
#X_train = pd.read_csv(baseFolder+"train/allData-classification-numeric-normalized_balanced_undersample.csv")
#X_train  = pd.read_csv(baseFolder+"test/allData-classification-numeric-normalized_balanced_oversample.csv")
X_train  = pd.read_csv(baseFolder+"test/allData-classification-numeric-normalized_balanced_oversample_smote.csv")

#AA = pd.read_csv(baseFolder+"allData-classification-numeric-normalized.csv")
#X_train, X_test = train_test_split(AA,test_size=0.25)

In [5]:
print(X_train.info())
X_train

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 191398 entries, 0 to 191397
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   activity            191398 non-null  float64
 1   location            191398 non-null  float64
 2   day_of_week         191398 non-null  float64
 3   light               191398 non-null  float64
 4   phone_lock          191398 non-null  float64
 5   proximity           191398 non-null  float64
 6   sound               191398 non-null  float64
 7   time_to_next_alarm  191398 non-null  float64
 8   minutes_day         191398 non-null  float64
 9   class               191398 non-null  object 
dtypes: float64(9), object(1)
memory usage: 14.6+ MB
None


,activity,location,day_of_week,light,phone_lock,proximity,sound,time_to_next_alarm,minutes_day,class
0,0.00,1.0,1.000000,0.0,0.0,1.0,0.655565,0.906151,0.115358,asleep
1,0.25,1.0,1.000000,0.0,0.0,1.0,0.567296,0.906052,0.116053,asleep
2,0.25,1.0,1.000000,0.0,0.0,1.0,0.567296,0.906052,0.116053,asleep
3,0.25,1.0,1.000000,0.0,0.0,1.0,0.660604,0.905952,0.116748,asleep
4,0.25,1.0,1.000000,0.0,0.0,1.0,0.660604,0.905952,0.116748,asleep
...,...,...,...,...,...,...,...,...,...,...
191393,0.25,1.0,1.000000,0.0,1.0,1.0,0.777203,0.005987,0.461189,asleep
191394,0.25,1.0,1.000000,0.0,1.0,1.0,0.000000,0.017182,0.359143,asleep
191395,0.25,0.5,0.000000,0.0,0.0,1.0,0.564570,0.960827,0.691889,asleep
191396,0.25,1.0,0.833333,0.0,1.0,0.0,0.804258,0.001468,0.474613,asleep


In [6]:
# transforms the input data to float32
X_test = transform_data_type(X_test)

# transforms the input data to float32
X_train = transform_data_type(X_train)


# balance
#print(Counter(X_train['class']))
#smote = SMOTE(random_state = 32)
#X_train, y_train = smote.fit_resample(X_train[inputFeatures], X_train['class'])
#X_train['class'] = y_train
print(Counter(X_train['class']))


# transform output to one_hot_encoding for the testing dataset
X_test = transform_output_nominal_class_into_one_hot_encoding(X_test)

# transform output to one_hot_encoding for the testing dataset
X_train = transform_output_nominal_class_into_one_hot_encoding(X_train)


Counter({'asleep': 95699, 'awake': 95699})


In [7]:
# selects the data to train and test
X_train_data = pd.DataFrame(data=X_train,columns=inputFeatures)
y_train_data = pd.DataFrame(data=X_train,columns=outputClasses)
# selec test dataset (fixed to all)
X_test_data = pd.DataFrame(data=X_test,columns=inputFeatures)
y_test_data = pd.DataFrame(data=X_test,columns=outputClasses)

X_train_data, y_train_data = create_dataset_time_series_with_one_output(   #timestamp
    X_train_data, 
    y_train_data, 
    TIME_SERIES_SIZE, 
    TIME_STEP_SHIFT
)

X_test_data, y_test_data = create_dataset_time_series_with_one_output(    #timestamp
    X_test_data, 
    y_test_data, 
    TIME_SERIES_SIZE, 
    TIME_STEP_SHIFT
)


print("shape: ",X_train_data.shape, y_train_data.shape)
print("Size: ",X_test_data.shape,y_test_data.shape)       

shape:  (191394, 4, 9) (191394, 2)
Size:  (136282, 4, 9) (136282, 2)


In [8]:
# transtorm data to tensor slices
test_dataset_series = tf.data.Dataset.from_tensor_slices((X_test_data, y_test_data))
train_dataset_series = tf.data.Dataset.from_tensor_slices((X_train_data, y_train_data))

2023-03-13 22:31:41.297162: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-13 22:31:41.355851: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-13 22:31:41.356221: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-13 22:31:41.357736: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [9]:
train_dataset_series


<TensorSliceDataset element_spec=(TensorSpec(shape=(4, 9), dtype=tf.float64, name=None), TensorSpec(shape=(2,), dtype=tf.uint8, name=None))>

In [10]:
#client_test_dataset.window(size=4, shift=1, stride=1, drop_remainder=True)
#test_dataset_series.batch(BATCH_SIZE) # usado no federated learning

In [11]:
# batch data size
#test_dataset_series1 = test_dataset_series.batch(1)
#train_dataset_series1 = train_dataset_series.batch(1)

test_dataset_series1 = test_dataset_series.batch(BATCH_SIZE)
train_dataset_series1 = train_dataset_series.batch(BATCH_SIZE)

train_dataset_series1

<BatchDataset element_spec=(TensorSpec(shape=(None, 4, 9), dtype=tf.float64, name=None), TensorSpec(shape=(None, 2), dtype=tf.uint8, name=None))>

In [12]:
print("input_shape=[", X_train_data.shape[1], X_train_data.shape[2],"]")
print("output shape:",len(outputClasses))

verbose, epochs, batch_size = 1, 2, BATCH_SIZE
#verbose, epochs, batch_size = 1, 3, 16
model = keras.Sequential()
model.add(LSTM(128,activation="tanh", 
          input_shape=[X_train_data.shape[1], X_train_data.shape[2]]))
#model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(len(outputClasses), activation='softmax'))#softmax,sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy     
# fit network
#model.fit(train_dataset_series1, epochs=epochs, verbose=verbose) #, batch_size=batch_size, validation_split=0.1
model.fit(X_train_data, y_train_data, epochs=epochs, verbose=verbose, batch_size=batch_size) #, batch_size=batch_size, validation_split=0.1
# evaluate model
#accuracy = model.evaluate(X_test_data, y_test_data) # , batch_size=batch_size, verbose=0
#accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
# predict
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

input_shape=[ 4 9 ]
output shape: 2


2023-03-13 22:31:49.269367: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 27560736 exceeds 10% of free system memory.


Epoch 1/2


2023-03-13 22:32:20.577193: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201
2023-03-13 22:32:31.769541: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


5982/5982 [==============================] - 91s 8ms/step - loss: 0.4227 - categorical_accuracy: 0.8031
Epoch 2/2
4259/4259 [==============================] - 11s 2ms/step

awake
Accuracy: 0.782583
Precision: 0.814686
Recall: 0.893184
F1 score: 0.852131
Cohens kappa: 0.444907
ROC AUC: 0.828295
\Confusion Matrix
[[21277 19420]
 [10210 85375]]

asleep
Accuracy: 0.782913
Precision: 0.672786
Recall: 0.523501
F1 score: 0.588829
Cohens kappa: 0.444457
ROC AUC: 0.828043
\Confusion Matrix
[[85513 10303]
 [19282 21184]]

Global
2
accuracy:  0.7827482719654834
precision:  0.7437357046088131
recall:  0.7083426439463123
f1_score:  0.720479885002507
cohen_kappa_score:  0.4446823647952441
roc_auc_score:  0.828168743046872


In [13]:
print("input_shape=[", X_train_data.shape[1], X_train_data.shape[2],"]")
print("output shape:",len(outputClasses))

verbose, epochs, batch_size = 1, 7, BATCH_SIZE
#verbose, epochs, batch_size = 1, 3, 16
model = keras.Sequential()
model.add(LSTM(128,activation="tanh", 
          input_shape=[X_train_data.shape[1], X_train_data.shape[2]]))
#model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(len(outputClasses), activation='softmax'))#softmax,sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy     
# fit network
#model.fit(train_dataset_series1, epochs=epochs, verbose=verbose) #, batch_size=batch_size, validation_split=0.1
model.fit(X_train_data, y_train_data, epochs=epochs, verbose=verbose, batch_size=batch_size) #, batch_size=batch_size, validation_split=0.1
# evaluate model
#accuracy = model.evaluate(X_test_data, y_test_data) # , batch_size=batch_size, verbose=0
#accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
# predict
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

input_shape=[ 4 9 ]
output shape: 2
Epoch 1/7
5982/5982 [==============================] - 48s 8ms/step - loss: 0.4255 - categorical_accuracy: 0.8026
Epoch 2/7
5982/5982 [==============================] - 51s 9ms/step - loss: 0.3613 - categorical_accuracy: 0.8339
Epoch 3/7
5982/5982 [==============================] - 53s 9ms/step - loss: 0.3321 - categorical_accuracy: 0.8444
Epoch 4/7
5982/5982 [==============================] - 50s 8ms/step - loss: 0.3093 - categorical_accuracy: 0.8542
Epoch 5/7
5982/5982 [==============================] - 50s 8ms/step - loss: 0.2895 - categorical_accuracy: 0.8630
Epoch 6/7
5982/5982 [==============================] - 47s 8ms/step - loss: 0.2741 - categorical_accuracy: 0.8713
Epoch 7/7
4259/4259 [==============================] - 12s 3ms/step

awake
Accuracy: 0.842716
Precision: 0.895332
Recall: 0.878443
F1 score: 0.886807
Cohens kappa: 0.629242
ROC AUC: 0.914483
\Confusion Matrix
[[30881  9816]
 [11619 83966]]

asleep
Accuracy: 0.842709
Precision: 0.

In [14]:
print("input_shape=[", X_train_data.shape[1], X_train_data.shape[2],"]")
print("output shape:",len(outputClasses))

verbose, epochs, batch_size = 1, 15, BATCH_SIZE
#verbose, epochs, batch_size = 1, 3, 16
model = keras.Sequential()
model.add(LSTM(128,activation="tanh", 
          input_shape=[X_train_data.shape[1], X_train_data.shape[2]]))
#model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(len(outputClasses), activation='softmax'))#softmax,sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy     
# fit network
#model.fit(train_dataset_series1, epochs=epochs, verbose=verbose) #, batch_size=batch_size, validation_split=0.1
model.fit(X_train_data, y_train_data, epochs=epochs, verbose=verbose, batch_size=batch_size) #, batch_size=batch_size, validation_split=0.1
# evaluate model
#accuracy = model.evaluate(X_test_data, y_test_data) # , batch_size=batch_size, verbose=0
#accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
# predict
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

input_shape=[ 4 9 ]
output shape: 2
Epoch 1/15
5982/5982 [==============================] - 52s 8ms/step - loss: 0.4258 - categorical_accuracy: 0.8003
Epoch 2/15
5982/5982 [==============================] - 51s 8ms/step - loss: 0.3616 - categorical_accuracy: 0.8331
Epoch 3/15
5982/5982 [==============================] - 48s 8ms/step - loss: 0.3327 - categorical_accuracy: 0.8446
Epoch 4/15
5982/5982 [==============================] - 50s 8ms/step - loss: 0.3094 - categorical_accuracy: 0.8550
Epoch 5/15
5982/5982 [==============================] - 52s 9ms/step - loss: 0.2928 - categorical_accuracy: 0.8630
Epoch 6/15
5982/5982 [==============================] - 51s 9ms/step - loss: 0.2781 - categorical_accuracy: 0.8692
Epoch 7/15
5982/5982 [==============================] - 50s 8ms/step - loss: 0.2653 - categorical_accuracy: 0.8755
Epoch 8/15
5982/5982 [==============================] - 50s 8ms/step - loss: 0.2526 - categorical_accuracy: 0.8812
Epoch 9/15
5982/5982 [======================

In [15]:
print("input_shape=[", X_train_data.shape[1], X_train_data.shape[2],"]")
print("output shape:",len(outputClasses))

verbose, epochs, batch_size = 1, 15, BATCH_SIZE
#verbose, epochs, batch_size = 1, 3, 16
model = keras.Sequential()
model.add(LSTM(128,activation="tanh", 
          input_shape=[X_train_data.shape[1], X_train_data.shape[2]]))
#model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(len(outputClasses), activation='softmax'))#softmax,sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy     
# fit network
#model.fit(train_dataset_series1, epochs=epochs, verbose=verbose) #, batch_size=batch_size, validation_split=0.1
model.fit(X_train_data, y_train_data, epochs=epochs, verbose=verbose, batch_size=batch_size) #, batch_size=batch_size, validation_split=0.1
# evaluate model
#accuracy = model.evaluate(X_test_data, y_test_data) # , batch_size=batch_size, verbose=0
#accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
# predict
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

input_shape=[ 4 9 ]
output shape: 2
Epoch 1/15
 221/5982 [>.............................] - ETA: 46s - loss: 0.6351 - categorical_accuracy: 0.6499

KeyboardInterrupt: 

In [18]:
print("input_shape=[", X_train_data.shape[1], X_train_data.shape[2],"]")
print("output shape:",len(outputClasses))

verbose, epochs, batch_size = 1, 100, BATCH_SIZE
#verbose, epochs, batch_size = 1, 3, 16
model = keras.Sequential()
model.add(LSTM(128,activation="tanh", 
          input_shape=[X_train_data.shape[1], X_train_data.shape[2]]))
#model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dropout(rate=0.5))
model.add(keras.layers.Dense(len(outputClasses), activation='softmax'))#softmax,sigmoid
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[tf.keras.metrics.CategoricalAccuracy()])
          #loss='binary_crossentropy',loss='categorical_crossentropy',
          #loss='binary_crossentropy',  sparse_categorical_crossentropy     
# fit network
model.fit(train_dataset_series1, epochs=epochs, verbose=verbose, batch_size=batch_size) #, batch_size=batch_size, validation_split=0.1
# evaluate model
#accuracy = model.evaluate(test_dataset_series1) # , batch_size=batch_size, verbose=0
# predict
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

input_shape=[ 4 9 ]
output shape: 2
Epoch 1/100
6896/6896 [==============================] - 53s 7ms/step - loss: 0.2948 - categorical_accuracy: 0.8796
Epoch 2/100
6896/6896 [==============================] - 50s 7ms/step - loss: 0.2989 - categorical_accuracy: 0.8755
Epoch 3/100
6896/6896 [==============================] - 49s 7ms/step - loss: 0.3014 - categorical_accuracy: 0.8727
Epoch 4/100
6896/6896 [==============================] - 59s 9ms/step - loss: 0.3000 - categorical_accuracy: 0.8781
Epoch 5/100
6896/6896 [==============================] - 63s 9ms/step - loss: 0.2947 - categorical_accuracy: 0.8804
Epoch 6/100
6896/6896 [==============================] - 64s 9ms/step - loss: 0.2932 - categorical_accuracy: 0.8833
Epoch 7/100
6896/6896 [==============================] - 63s 9ms/step - loss: 0.2925 - categorical_accuracy: 0.8837
Epoch 8/100
6896/6896 [==============================] - 63s 9ms/step - loss: 0.2858 - categorical_accuracy: 0.8883
Epoch 9/100
6896/6896 [=============

In [20]:
yhat_probs = model.predict(X_test_data)
# predict crisp classes for test set deprecated

probss = pd.DataFrame(data=yhat_probs,columns=['awake','asleep'])
valuesY = pd.DataFrame(data=y_test_data,columns=['awake','asleep'])

#valuesY = y_test_data
print('')
print('awake')
test = list()
res = printMetrics(valuesY['awake'],probss['awake'])
test.append(res)
print('')
print('asleep')
res = printMetrics(valuesY['asleep'],probss['asleep'])
test.append(res)
print('')
print('Global')
showGlobalMetrics(test)

4259/4259 [==============================] - 11s 2ms/step

awake
Accuracy: 0.272685
Precision: 0.979019
Recall: 0.102880
F1 score: 0.186194
Cohens kappa: 0.038272
ROC AUC: 0.737497
\Confusion Matrix
[[25823   243]
 [98877 11339]]

asleep
Accuracy: 0.270953
Precision: 0.205180
Recall: 0.990630
F1 score: 0.339950
Cohens kappa: 0.037826
ROC AUC: 0.706877
\Confusion Matrix
[[11340 99114]
 [  242 25586]]

Global
2
accuracy:  0.2718187288123156
precision:  0.5920998003566356
recall:  0.5467550616729189
f1_score:  0.263071788036283
cohen_kappa_score:  0.03804903634117718
roc_auc_score:  0.7221871440907924
